In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys
sys.path.append(os.environ['GORDON_REPO_DIR'] + '/pipeline_scripts')

from utilities2014 import *

sys.path.append('/home/yuncong/project/opencv-2.4.9/release/lib/python2.7/site-packages')
import cv2

from joblib import Parallel, delayed

from collections import defaultdict, Counter
from itertools import combinations, chain, product

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=1)
dm._load_image()

texton_hists = dm.load_pipeline_result('texHist', 'npy')
segmentation = dm.load_pipeline_result('segmentation', 'npy')
n_superpixels = len(np.unique(segmentation)) - 1
textonmap = dm.load_pipeline_result('texMap', 'npy')
n_texton = len(np.unique(textonmap)) - 1
neighbors = dm.load_pipeline_result('neighbors', 'pkl')
sp_properties = dm.load_pipeline_result('spProps', 'npy')
segmentation_vis = dm.load_pipeline_result('segmentationWithText', 'jpg')

# texture_map = texton_hists[segmentation]
# texture_map[~dm.mask] = np.nan * np.ones((n_texton,))
# dm.save_pipeline_result(texture_map, 'textureMap', 'npy')
texture_map = dm.load_pipeline_result('textureMap', 'npy')

coherence_map = dm.load_pipeline_result('coherenceMap', 'npy')
eigenvec_map = dm.load_pipeline_result('eigenvecMap', 'npy')

/oasis/projects/nsf/csd181/yuncong/virtualenv-1.9.1/yuncongve/lib/python2.7/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [3]:
import cPickle as pickle
from enum import Enum
from itertools import groupby
from operator import itemgetter

In [4]:
class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'

In [5]:
with open(os.environ['GORDON_LABELING_DIR']+'/RS141_0001_yuncong_08142015182314.pkl') as f:
    labelings = pickle.load(f)

In [6]:
thetas = np.linspace(-np.pi/4, np.pi/4, 9)
n_theta = len(thetas)
Rs = [np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]) for theta in thetas]

In [150]:
from scipy.spatial.distance import cdist

def vertices_to_normals(vertices):
    
    n = vertices.shape[0]

    # compute normal direction of each vertex
    D = cdist(vertices, vertices)
    nn = D.argsort(axis=1)[:,:3]
    vertice_normals = np.empty((n, 2))
    for i, neighborhood in enumerate(nn):
        X = vertices[neighborhood]
        Xc = X - X.mean(axis=0)
        U,S,V = np.linalg.svd(np.dot(Xc.T, Xc))
        tangent_dir = U[:,0]
        vertice_normals[i] = np.array([tangent_dir[1], -tangent_dir[0]])
        if vertice_normals[i][0] < 0: # make sure x-component is positive, i.e. normal points rightwards
            vertice_normals[i] = -vertice_normals[i]
    return vertice_normals


def find_in_polygon(vertices, x, y):

    from matplotlib.path import Path

    X, Y = np.meshgrid(x, y)  # X, Y are 2D ndarrays
    XY = np.dstack((X, Y))
    XY_flat = XY.reshape((-1, 2))

    mpath = Path(vertices) # the vertices of the polygon
    mask_flat = mpath.contains_points(XY_flat)
    mask = mask_flat.reshape(X.shape)

    return mask

    
def compute_landmark_descriptors(group):
    
    n_polygon = len(group)
        
    all_polygon_vertices = [vertices for _, _, vertices in group]
    all_polygon_types = [polygon_type for _, polygon_type, _ in group]
        
    edge_indices = [i for i in range(n_polygon)
                   if all_polygon_types[i] == PolygonType.TEXTURE_WITH_CONTOUR \
                            or all_polygon_types[i] == PolygonType.CLOSED\
                           or all_polygon_types[i] == PolygonType.OPEN]
    texture_indices = [i for i in range(n_polygon)
                   if all_polygon_types[i] == PolygonType.TEXTURE_WITH_CONTOUR \
                            or all_polygon_types[i] == PolygonType.TEXTURE]
    
    res = [dict([]) for _ in range(n_theta)]
        
    all_vertices = np.vstack(all_polygon_vertices)
    
    xmin, ymin = all_vertices.min(axis=0).astype(np.int)
    xmax, ymax = all_vertices.max(axis=0).astype(np.int)
    centroid_global = all_vertices.mean(axis=0).astype(np.int)
    centroid_local = centroid_global - [xmin, ymin]
    lm_texture_template = texture_map[ymin:ymax+1, xmin:xmax+1]

    lm_box_shape = [xmax - xmin + 1, ymax - ymin + 1]

    texture_sample_radius = 5
    int_rs = np.arange(-texture_sample_radius, 0)
    ext_rs = np.arange(1, texture_sample_radius+1)
    
    all_polygon_vertice_normals = map(vertices_to_normals, all_polygon_vertices)
    
    striation_indices = [i for i in range(n_polygon) if all_polygon_types[i] == PolygonType.DIRECTION]
    if len(striation_indices) > 0:
        striation_sample_points = np.vstack(all_polygon_vertices[i] for i in striation_indices).astype(np.int)
        striation_sample_vecs = eigenvec_map[striation_sample_points[:,1], striation_sample_points[:,0]]
    
    for theta_i in range(n_theta):
        
#         print 'theta', theta_i
        
        all_polygon_vertices_rotated_global_list = []
        
        for vertices in all_polygon_vertices:
            vertices_centered = vertices - centroid_global
            vertices_centered_rotated = np.dot(Rs[theta_i], vertices_centered.T).astype(np.int).T
            vertices_rotated_global = vertices_centered_rotated + centroid_global
            all_polygon_vertices_rotated_global_list.append(vertices_rotated_global)
            
        all_polygon_vertices_rotated_global = np.vstack(all_polygon_vertices_rotated_global_list)
        all_vertices_rotated_global_xmin, all_vertices_rotated_global_ymin = all_polygon_vertices_rotated_global.min(axis=0)
        all_vertices_rotated_global_xmax, all_vertices_rotated_global_ymax = all_polygon_vertices_rotated_global.max(axis=0)
        centroid_rotated_local = centroid_global - (all_vertices_rotated_global_xmin, all_vertices_rotated_global_ymin)
        
        rotated_bbox_shape = (all_vertices_rotated_global_xmax - all_vertices_rotated_global_xmin + 1, 
                                all_vertices_rotated_global_ymax - all_vertices_rotated_global_ymin + 1)

        res[theta_i]['bbox'] = np.array([rotated_bbox_shape[0], rotated_bbox_shape[1], 
                                 centroid_rotated_local[0], centroid_rotated_local[1],
                                        centroid_global[0], centroid_global[1],
                                        xmin, ymin, xmax, ymax])
        
        all_polygon_vertices_rotated_local = [vertices_rotated_global - (all_vertices_rotated_global_xmin, all_vertices_rotated_global_ymin)
                                  for vertices_rotated_global in all_polygon_vertices_rotated_global_list]
        
        
        
        res[theta_i]['vertices'] = np.vstack(all_polygon_vertices_rotated_local)
        res[theta_i]['vertices_global'] = np.vstack(all_polygon_vertices_rotated_global_list)

        
        if len(edge_indices) > 0:
            all_polygon_vertice_normals_rotated = []
            for vertice_normals in all_polygon_vertice_normals:
                vertice_normals_rotated = np.dot(Rs[theta_i], vertice_normals.T).T
                all_polygon_vertice_normals_rotated.append(vertice_normals_rotated)
            res[theta_i]['normal'] = np.vstack([all_polygon_vertice_normals_rotated[i] for i in edge_indices])
        else:
            res[theta_i]['normal'] = None

        if len(edge_indices) > 0:

            all_polygon_boundary_int_textures = []
            all_polygon_boundary_ext_textures = []
            for vertices, vertice_normals in zip(all_polygon_vertices, all_polygon_vertice_normals):

                int_texture_sample_xs = (vertices[:,0][:,None] + np.outer(vertice_normals[:,0], int_rs)).astype(np.int)
                int_texture_sample_ys = (vertices[:,1][:,None] + np.outer(vertice_normals[:,1], int_rs)).astype(np.int)
                boundary_int_textures = texture_map[int_texture_sample_ys, int_texture_sample_xs].mean(axis=1)
                all_polygon_boundary_int_textures.append(boundary_int_textures)

                ext_texture_sample_xs = (vertices[:,0][:,None] + np.outer(vertice_normals[:,0], ext_rs)).astype(np.int)
                ext_texture_sample_ys = (vertices[:,1][:,None] + np.outer(vertice_normals[:,1], ext_rs)).astype(np.int)
                boundary_ext_textures = texture_map[ext_texture_sample_ys, ext_texture_sample_xs].mean(axis=1)
                all_polygon_boundary_ext_textures.append(boundary_ext_textures)
        
            res[theta_i]['boundary_vertices'] = np.vstack(all_polygon_vertices_rotated_local[i] for i in edge_indices)
            res[theta_i]['boundary_vertices_global'] = np.vstack(all_polygon_vertices_rotated_global_list[i] for i in edge_indices)
            res[theta_i]['boundary_texture'] = np.c_[np.vstack(all_polygon_boundary_int_textures[i] for i in edge_indices), 
                                                     np.vstack(all_polygon_boundary_ext_textures[i] for i in edge_indices)]
        
        else:
            res[theta_i]['boundary_vertices'] = None
            res[theta_i]['boundary_vertices_global'] = None
            res[theta_i]['boundary_texture'] = None
            
        
        if len(striation_indices) > 0:
            striation_sample_points_rotated_local = np.vstack(all_polygon_vertices_rotated_local[i] for i in striation_indices).astype(np.int)
            striation_sample_vecs_rotated = np.dot(Rs[theta_i], striation_sample_vecs.T).T
            res[theta_i]['striation'] = np.c_[striation_sample_points_rotated_local, striation_sample_vecs_rotated]
            res[theta_i]['striation_points_global'] = np.vstack(all_polygon_vertices_rotated_global_list[i] for i in striation_indices)
        else:
            res[theta_i]['striation'] = None
            res[theta_i]['striation_points_global'] = None
            
        # rotated templates
        rotated_texTemplate = np.nan * np.ones((rotated_bbox_shape[1], rotated_bbox_shape[0], n_texton))
                       
#         for vertices in all_polygon_vertices_rotated_local:   
#             mask = find_in_polygon(vertices, range(rotated_bbox_shape[0]), range(rotated_bbox_shape[1]))
#             plt.imshow(mask)
#             plt.show()
                           
        ys, xs = np.mgrid[:rotated_bbox_shape[1], :rotated_bbox_shape[0]]
        a = np.dot(np.linalg.inv(Rs[theta_i]), (np.c_[xs.flat, ys.flat] - centroid_rotated_local).T).T

        xss = (a[:,0] + centroid_local[0]).astype(np.int)
        yss = (a[:,1] + centroid_local[1]).astype(np.int)
        
        valid = (yss < lm_texture_template.shape[0]) & (yss >= 0) & (xss < lm_texture_template.shape[1]) & (xss >= 0)
        rotated_texTemplate[ys.flat[valid], xs.flat[valid]] = lm_texture_template[yss[valid], xss[valid]]

        
        if len(texture_indices) > 0:
            all_polygon_masks = [find_in_polygon(all_polygon_vertices_rotated_local[i], range(rotated_bbox_shape[0]), range(rotated_bbox_shape[1]))
                                 for i in texture_indices]
            
            if len(all_polygon_masks) == 1:
                rotated_texTemplate[~all_polygon_masks[0]] = np.nan
            else:
                rotated_texTemplate[~np.bitwise_or(*all_polygon_masks)] = np.nan
                
            res[theta_i]['texture_template'] = rotated_texTemplate
            
            textured_ys, textured_xs = np.where(~np.isnan(rotated_texTemplate[:,:,0]))
            textured_ys_global = textured_ys + all_vertices_rotated_global_ymin
            textured_xs_global = textured_xs + all_vertices_rotated_global_xmin
            res[theta_i]['texture_vertices_global'] = np.c_[textured_xs_global, textured_ys_global]
        else:
            res[theta_i]['texture_template'] = None
            res[theta_i]['texture_vertices_global'] = None
            
    return res
                    
#         fig = plt.figure()
#         ax = fig.add_subplot(111)
# #             q = np.zeros_like(dm.image)
# #             q[vertices_rotated_global[:,1], vertices_rotated_global[:,0]] = 1
# #             plt.imshow(q)
# #         all_polygon_vertices_rotated_global = np.vstack(all_polygon_vertices_rotated_global)

#         ax.scatter(all_polygon_vertices_rotated_global[:,0], all_polygon_vertices_rotated_global[:,1])
#         ax.set_aspect('equal')
# #         ax.set_xlim([2800,3300])
# #         ax.set_ylim([1900,2300])
#         plt.show()

In [63]:
landmark_descriptor = compute_landmark_descriptors([labelings['final_polygons'][5]])

In [151]:
def q(label, group):
    descriptor = compute_landmark_descriptors(list(group))
    with open('/home/yuncong/csd395/all_landmark_descriptors_%d.pkl'%label, 'w') as f:
        pickle.dump(descriptor, f)
    return label, descriptor
    
landmark_descriptors = dict(Parallel(n_jobs=16)(delayed(q)(label, list(group)) 
                        for label, group in groupby(labelings['final_polygons'], itemgetter(0))))

In [41]:
# for label, group in groupby(labelings['final_polygons'], itemgetter(0)):
#     print label
#     descriptor = compute_landmark_descriptors(list(group))
#     with open('/home/yuncong/csd395/all_landmark_descriptors_%d.pkl'%label, 'w') as f:
#         pickle.dump(descriptor, f)

0
7
5
8
6
9
10
11
12
13


In [112]:
from skimage.morphology import disk, binary_dilation

viz_overlay = np.zeros_like(dm.image_rgb, np.uint8)

for label, descriptor in landmark_descriptors.iteritems():
    print label
    
    pts = descriptor[4]['boundary_vertices_global']
#     c = (255,0,0)
    if pts is not None:
        overlay = np.zeros_like(dm.image, np.bool)
        overlay[pts[:,1], pts[:,0]] = 1
        overlay = binary_dilation(overlay, disk(10))
#         viz[overlay] = c
        viz_overlay[overlay] = (255,0,0)
    
    pts = descriptor[4]['texture_vertices_global']
#     c = (0,255,0)
    if pts is not None:
#         viz[pts[:,1], pts[:,0]] = c
        viz_overlay[pts[:,1], pts[:,0]] = (0,255,0)
    
    pts = descriptor[4]['striation_points_global']
    if pts is not None:
        vecs = descriptor[4]['striation'][:, 2:]
        
        overlay = np.zeros_like(dm.image, np.bool)
        overlay[pts[:,1], pts[:,0]] = 1
        overlay = binary_dilation(overlay, disk(5))
#         c = (255,255,0)
#         viz[overlay] = c
        
        length = 50.
        ends = (pts + length * np.c_[vecs[:,1], -vecs[:,0]]).astype(np.int)
        for end, (x,y) in zip(ends, pts):
            cv2.line(viz_overlay, (x,y), tuple(end), (255,0,0), thickness=5, lineType=8, shift=0)
        viz_overlay[overlay] = (255,255,0)

0
5
6
7
8
9
10
11
12
13


In [146]:
vis_overlay_alpha = np.ones(viz_overlay.shape[:2])
vis_overlay_alpha[np.all(viz_overlay == 0, axis=-1)] = 0.

viz = alpha_blending(dm.image_rgb, img_as_float(viz_overlay), .3, vis_overlay_alpha)
# viz = alpha_blending(np.ones_like(dm.image_rgb), img_as_float(viz_overlay), .3, vis_overlay_alpha)
display(viz)

/oasis/projects/nsf/csd395/yuncong/Brain/notebooks/tmp.jpg

In [22]:
[[rotated_version['bbox'] for rotated_version in all_landmark_descriptors[0]]]

[[array([397, 421, 193, 174]),
  array([433, 375, 197, 155]),
  array([460, 325, 200, 133]),
  array([478, 286, 195, 121]),
  array([479, 273, 184, 114]),
  array([485, 277, 189, 105]),
  array([475, 316, 189, 138]),
  array([452, 361, 185, 174]),
  array([421, 397, 174, 203])]]